In [1]:
## 使用Qwen-turbo，对保险客户数据表进行SQL查询

In [1]:
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role
import time
import pandas as pd
import re
# 从环境变量获取 dashscope 的 API Key
api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = api_key

# 封装模型响应函数
def get_response(messages):
    response = dashscope.Generation.call(
        model='qwen-turbo-latest',
        messages=messages,
        result_format='message'  # 将输出设置为message形式
    )
    return response

# 从模型响应中提取SQL代码
def get_sql_code(response):
    # 查找```sql和```之间的内容
    pattern = r'```sql(.*?)```'
    match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # 如果没有找到```sql标记，尝试查找任何```之间的内容
        pattern = r'```(.*?)```'
        match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            # 如果没有找到任何代码块，返回整个响应
            return response.output.choices[0].message.content

# 得到sql
def get_sql(query, table_description):
    start_time = time.time()
    sys_prompt = """我正在编写SQL，以下是数据库中的数据表和字段，请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL，如果有多个查询语句，请尝试合并为一个。编写SQL请采用```sql
    """
    user_prompt = f"""{table_description}
=====
我要写的SQL是：{query}
请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL
"""
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = get_response(messages)
    return response

########## 以下参数需要进行确认 ##########
save_file = f'sql_result_qwen_turbo.xlsx'
data_file = './data/数据表字段说明-精简1.txt'
qa_file = './qa_list-2.txt'

In [2]:
# 读取 数据表字段说明-精简
with open(data_file, 'r', encoding='utf-8') as file:
    table_description = file.read()
# 读取 SQL问题列表
with open(qa_file, 'r', encoding='utf-8') as file:
    qa_list = file.read()
qa_list = qa_list.split('=====')

# 保存SQL结果
sql_list = []
markdown_list = []
time_list = []
for qa in qa_list:
    query = qa
    query = query.replace('\n', '')
    print(query)
    start_time = time.time()
    # 请求生成sql
    response = get_sql(query, table_description)
    use_time = round(time.time()-start_time, 2)
    time_list.append(use_time)
    print('SQL生成时间：', use_time)
    print('response=', response.output.choices[0].message.content)
    # 提取生成的SQL
    sql = get_sql_code(response)
    print('SQL: {}'.format(sql))
    sql_list.append(sql)

获取所有客户的姓名和联系电话。
SQL生成时间： 2.16
response= 根据你的需求：**获取所有客户的姓名和联系电话**，我们只需要从 **客户信息表（CustomerInfo）** 中查询对应的字段即可。

### 涉及的数据表和字段：
- **表名**：`CustomerInfo`
- **字段**：
  - `Name`（客户姓名）
  - `PhoneNumber`（联系电话）

### SQL语句如下：

```sql
SELECT Name, PhoneNumber
FROM CustomerInfo;
```

✅ 这个查询简洁高效，仅使用了必要的表和字段，满足你“获取所有客户的姓名和联系电话”的需求。
SQL: SELECT Name, PhoneNumber
FROM CustomerInfo;
找出所有已婚客户的客户ID和配偶姓名。
SQL生成时间： 7.07
response= 要找出所有**已婚客户的客户ID和配偶姓名**，我们需要分析如下：

---

### ✅ 需要的数据表及字段：
1. **CustomerInfo 表**  
   - `CustomerID`：用于标识客户  
   - `MaritalStatus`：判断是否已婚（如值为 "已婚" 或 "Married"）  
   - `Name`：客户姓名（但题目要求的是配偶姓名，不是客户本人）

2. **BeneficiaryInfo 表**  
   - `BeneficiaryID`：受益人唯一标识  
   - `Name`：受益人姓名（可能就是配偶）  
   - `Relationship`：关系字段（例如“配偶”、“Spouse”等）  
   - `CustomerID`：关联到客户（通过这个可以知道谁的配偶）

> 💡 关键逻辑：  
在保单信息中，受益人关系字段（`Relationship`）常用来记录“配偶”，且受益人信息表中包含姓名。  
所以我们可以从 **CustomerInfo** 中筛选出婚姻状态为“已婚”的客户，并通过 **BeneficiaryInfo** 找到其对应的配偶（即关系为“配偶”的受益人）。

---

### 🧠 注意事项：
- 并非每个客户都有明确的“配偶”作为受益人，因此需要 LEFT JOIN。
- 假设一个

In [3]:
result = pd.DataFrame(columns=['QA', 'SQL', 'time'])
result['QA'] = qa_list
result['SQL'] = sql_list
result['time'] = time_list
result.to_excel(save_file, index=False)
result

,QA,SQL,time
0,获取所有客户的姓名和联系电话。\n,"SELECT Name, PhoneNumber\nFROM CustomerInfo;",2.16
1,\n找出所有已婚客户的客户ID和配偶姓名。\n,"SELECT \n c.CustomerID,\n b.Name AS Spou...",7.07
2,\n查询所有未支付保费的保单号和客户姓名。\n,"SELECT \n p.PolicyNumber AS 保单号,\n c.Nam...",7.64
3,\n找出所有理赔金额大于10000元的理赔记录，并列出相关客户的姓名和联系电话。\n,"SELECT \n c.Name AS 客户姓名,\n c.PhoneNumbe...",6.23
4,\n查找代理人的姓名和执照到期日期，按照执照到期日期升序排序。\n,"SELECT \n Name AS 代理人姓名,\n LicenseExpira...",2.80
5,\n获取所有保险产品的产品名称和保费，按照保费降序排序。\n,"SELECT ProductName, Premium\nFROM ProductInfo\...",2.78
6,\n查询销售区域在上海的员工，列出他们的姓名和职位。\n,"SELECT \n Name AS 姓名,\n Position AS 职位\n...",5.36
7,\n找出所有年龄在30岁以下的客户，并列出其客户ID、姓名和出生日期。\n,"DATEDIFF(CURDATE(), DateOfBirth) / 365.25 < 30",4.49
8,\n查找所有已审核但尚未支付的理赔记录，包括理赔号、审核人和审核日期。\n,"SELECT \n ClaimNumber AS 理赔号,\n ClaimHan...",5.37
9,\n获取每个产品类型下的平均保费，以及该产品类型下的产品数量。,"SELECT \n ProductType AS 产品类型,\n AVG(Pre...",3.66
